# Exercise: Generating one token at a time

In this exercise, we will get to understand how an LLM generates text--one token at a time, using the previous tokens to predict the following ones.


## Step 1. Load a tokenizer and a model

First we load a tokenizer and a model from HuggingFace's transformers library. A tokenizer is a function that splits a string into a list of numbers that the model can understand.

In this exercise, all the code will be written for you. All you need to do is follow along!

In [1]:
# Student Task: Load the tokenizer and model for `gpt2` from HuggingFace
# TODO: Replace the ********** with the correct code

from transformers import AutoModelForCausalLM, AutoTokenizer

# To load a pretrained model and a tokenizer using HuggingFace, we only need two lines of code!
tokenizer = AutoTokenizer.from_pretrained('gpt2')
model = AutoModelForCausalLM.from_pretrained('gpt2')


In [2]:
# We create a partial sentence and tokenize it.
# No changes needed here

text = "Udacity is the best place to learn about generative"
inputs = tokenizer(text, return_tensors="pt")  # pt is for pytorch; tf is for tensorflow

# Show the tokens as numbers, i.e. "input_ids"
inputs["input_ids"]

tensor([[  52,   67, 4355,  318,  262, 1266, 1295,  284, 2193,  546, 1152,  876]])

## Step 2. Examine the tokenization

Let's explore what these tokens mean!

In [3]:
# Show how the sentence is tokenized
# No changes needed here

import pandas as pd


def show_tokenization(inputs):
    return pd.DataFrame(
        [(id, tokenizer.decode(id)) for id in inputs["input_ids"][0]],
        columns=["id", "token"],
    )


show_tokenization(inputs)

,id,token
0,tensor(52),U
1,tensor(67),d
2,tensor(4355),acity
3,tensor(318),is
4,tensor(262),the
5,tensor(1266),best
6,tensor(1295),place
7,tensor(284),to
8,tensor(2193),learn
9,tensor(546),about


### Subword tokenization

The interesting thing is that tokens in this case are neither just letters nor just words. Sometimes shorter words are represented by a single token, but other times a single token represents a part of a word, or even a single letter. This is called subword tokenization.

## Step 2. Calculate the probability of the next token

Now let's use PyTorch to calculate the probability of the next token given the previous ones.

In [4]:
# Calculate the probabilities for the next token for all possible choices. We show the
# top 5 choices and the corresponding words or subwords for these tokens.
# No changes needed here

import torch

with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)


def show_next_token_choices(probabilities, top_n=5):
    return pd.DataFrame(
        [
            (id, tokenizer.decode(id), p.item())
            for id, p in enumerate(probabilities)
            if p.item()
        ],
        columns=["id", "token", "p"],
    ).sort_values("p", ascending=False)[:top_n]


show_next_token_choices(probabilities)

,id,token,p
8300,8300,programming,0.157594
4673,4673,learning,0.148418
4981,4981,models,0.048508
17219,17219,biology,0.046481
16113,16113,algorithms,0.027796


Interesting! The model thinks that the most likely next word is "programming", followed up closely by "learning".

In [5]:
# Student task: Obtain the token id for the most probable next token
# TODO: Replace the ********** with the correct code

# Get the id of the most probable next token, i.e. the index of the highest value in `probabilities`
# Hint: torch.argmax will be useful
# Hint: `.item()` converts a single-value tensor to a standard Python number
next_token_id = torch.argmax(probabilities).item()


print(f"Next token id: {next_token_id}")
print(f"Next token: {tokenizer.decode(next_token_id)}")

Next token id: 8300
Next token:  programming


In [6]:
# We append the most likely token to the text.
# No changes needed here

new_text = text + tokenizer.decode(8300)
new_text

'Udacity is the best place to learn about generative programming'

## Step 3. Generate some more tokens

The following cell will take `text`, show the most probable tokens to follow, and append the most likely token to text. Run the cell over and over to see it in action!

In [7]:
# Press ctrl + enter to run this cell again and again to see how the text is generated.

from IPython.display import Markdown, display

# Show the text
print(new_text)

# Convert to tokens
inputs = tokenizer(text, return_tensors="pt")

# Calculate the probabilities for the next token and show the top 5 choices
with torch.no_grad():
    logits = model(**inputs).logits[:, -1, :]
    probabilities = torch.nn.functional.softmax(logits[0], dim=-1)

display(Markdown("**Next token probabilities:**"))
display(show_next_token_choices(probabilities))

# Choose the most likely token id and add it to the text
next_token_id = torch.argmax(probabilities).item()
new_text = new_text + tokenizer.decode(next_token_id)

Udacity is the best place to learn about generative programming


**Next token probabilities:**

,id,token,p
8300,8300,programming,0.157594
4673,4673,learning,0.148418
4981,4981,models,0.048508
17219,17219,biology,0.046481
16113,16113,algorithms,0.027796


## Step 4. Use the `generate` method

In [8]:
# Student Task: Use the model.generate method to generate lots of text quickly
# TODO: Replace the ********** with the correct code

from IPython.display import Markdown, display

# Start with some text and tokenize it
text = "Once upon a time, generative models"
inputs = tokenizer(text, return_tensors="pt")

# Use the `generate` method to generate a max of 100 tokens
# See: https://docs.pytorch.org/torchtune/0.3/generated/torchtune.generation.generate.html
# eos_token_id: "End Of Sentence." Every model has a special secret number that means "I'm done talking."
# pad_token_id: When the model finishes early, or if we are processing multiple sentences of different lengths, we need to fill the empty space with "filler" numbers (padding).
# Why we set them equal: Some older models (like GPT-2) don't have a dedicated "padding" button. By telling the model to use the eos_token_id as the "filler," we are basically saying: "If you have extra space you need to fill, just use the 'Be Quiet' symbol." This prevents the model from generating random gibberish or throwing an error.
output = model.generate(**inputs, max_new_tokens=100, pad_token_id=tokenizer.eos_token_id)


# Show the generated text
display(Markdown(tokenizer.decode(output[0])))

Once upon a time, generative models of the human brain were used to study the neural correlates of cognitive function. In the present study, we used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human brain to investigate the neural correlates of cognitive function. We used a novel model of the human

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from IPython.display import Markdown, display

# Try another model
model_name = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

# Load the model in Float16 (Half precision)
# This uses ~16GB of your 32GB RAM, leaving the other 16GB for macOS and your UI.
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    dtype=torch.float16, 
    device_map="auto" # This will automatically pick your "mps" GPU
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Using device: ", model.device)

text = "Once upon a time, generative models"
inputs = tokenizer(text, return_tensors="pt").to(model.device)
output = model.generate(
    **inputs, 
    use_cache=True,
    max_new_tokens=100, 
    do_sample=True,  # Use 'do_sample=True' to enable temperature for better creativity
    temperature=0.7,  # temperature is to improve model's creativity instead of just picking the most likely tokens
    pad_token_id=tokenizer.eos_token_id
)  

# Show the generated text
display(Markdown(tokenizer.decode(output[0])))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Using device:  mps:0


<｜begin▁of▁sentence｜>Once upon a time, generative models like GPT-4 were considered state-of-the-art. But now, they are being surpassed by more efficient models, such as those based on the transformer architecture with fewer layers but larger widths and depths. However, the question is: What exactly makes a model more efficient? Is it just about processing speed, or is it also about other factors like memory usage, cost, and scalability? Let's delve into this.

First, let's define what efficiency means in the context of generative models.

### That's interesting...

You'll notice that GPT-2 is not nearly as sophisticated as later models like GPT-4/5. It often repeats itself and doesn't always make much sense. But it's still pretty impressive that it can generate text that looks like English.

## Congrats for completing the exercise! 🎉

Give yourself a hand. And please take a break if you need to. We'll be here when you're refreshed and ready to learn more!

<br /><br /><br /><br /><br /><br /><br /><br />